![Quantus Telematika Indonesia](../../images/logo-construction-small.png)

# Supervised Machine Learning - Classification

## Load Library

In [ ]:
# the big three
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# eli5
import eli5

# scikit-learn
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# yellowbrick
from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import PrecisionRecallCurve

## Load Dataset

In [ ]:
# membaca file csv
raw_data = pd.read_csv("./dataset/cancer/data.csv")

**Membaca Metadata**

In [ ]:
# membaca metadata
with open('./dataset/cancer/data_description.txt', 'r') as f:
    teks = f.readlines()
    
for tulisan in teks:
    print(tulisan)

In [ ]:
# melihat isi dari raw_data
raw_data

In [ ]:
# mengambil indeks kolom ke 1 sampai indeks paling akhir
kolom = raw_data.columns[1:]

# mengambil raw_data dari kolom yang sudah didefinisikan
raw_data = raw_data[kolom].copy()

In [ ]:
raw_data

**Inspeksi Data**

In [ ]:
# cek summary
raw_data.describe()

In [ ]:
# Cek Nilai yang hilang
raw_data.isna().sum()

In [ ]:
# buat variabel baru dengan diagnosis difokuskan pada apakah
# tumor termasuk ganas atau tidak ('malignant' atau 'not malignant')
raw_data['diagnosis'] = pd.get_dummies(raw_data['diagnosis'])['M']

In [ ]:
# cek class imbalance
raw_data['diagnosis'].value_counts().plot.bar();

**Train-Test Splitting**

In [ ]:
# train-test split
training_data = raw_data.groupby('diagnosis', group_keys=False).apply(lambda x: x.sample(frac = .8, random_state=1000)).copy()

training_data

In [ ]:
test_data = raw_data.drop(training_data.index)

test_data

In [ ]:
# buat train set
X_train = training_data.iloc[:,1:].values
y_train = training_data['diagnosis'].values

# buat validation/test set
X_test = test_data.iloc[:,1:].values
y_test = test_data['diagnosis'].values

## Bayes Classifier

Bayes Classifier (disebut juga *Naive Bayes Classifier*) adalah salah satu model machine learning yang mana menginsyaratkan tidak adanya keterkaitan antar fitur.

### Dasar Teori

Classifier Bayes dirumuskan berdasarkan Teorema Bayes berikut:

Misalkan terdapat dua kejadian $A$ dan $B$. Kita dapat mengkaitkan masing-masing probabilitas dari dua kejadian ini, $P(A)$ dan $P(B)$, dengan probabilitas kondisional $P(A|B)$ dan $P(B|A)$ berdasarkan aturan berikut:

$$
P(A \cap B) = P(A|B)P(B)
$$

atau

$$
P(B \cap A) = P(B|A)P(A)
$$

Karena

$$P(A \cap B) = P(B \cap A)$$

maka persamaan diatas dapat ditulis kembali sebagai:

$$
P(A|B) = \frac{P(B|A)P(A)}{P(B)}
$$

**Mengapa Menggunakan Naive Bayes**

- Cepat
- Hanya membutuhkan sedikit parameter
- Out-of-core learning

**Mengapa Tidak Menggunakan Naive Bayes**

- Base model

## Naive Bayes pada Scikit-Learn

Terdapat 3 implementasi Naive Bayes pada Scikit-Learn, diantaranya:

- Bernoulli Naive Bayes (```BernoulliNB```)
- Multinomial Naive Bayes (```MultinomialNB```)
- Gaussian Naive Bayes (```GaussianNB```)
- Complement Naive Bayes (```ComplementNB```)

### Bernoulli Naive Bayes

Bernoulli Naive Bayes digunakan jika sebaran data mengikuti distribusi Bernoulli.

**Kapan Menggunakan Bernoulli Naive Bayes**

- Ketika tiap fitur dalam matriks bernilai biner 0-1. Misalnya bernilai 0 jika kata tersebut tidak ada di dalam teks, dan 1 jika kata tersebut ada di dalam teks.
- Cocok untuk klasifikasi pesan singkat

### Multinomial Naive Bayes

Multinomial Naive Bayes digunakan untuk data yang sifatnya diskrit.

**Kapan Menggunakan Multinomial Naive Bayes**

- Ketika tiap fitur dalam matriks memiliki nilai diskrit, misalnya dari hasil counting.
- Cocok untuk klasifikasi teks

### Gaussian Naive Bayes

Gaussian Naive Bayes digunakan untuk data dengan fitur numerik.

**Kapan Menggunakan Gaussian Naive Bayes**

- Ketika tiap fitur dalam matriks memiliki nilai numerik

**Syntax GaussianNB**

```python
GaussianNB(priors=None, var_smoothing=1e-09)
```

Keterangan:

- ```priors```: Probabilitas prior dari kelas.
- ```var_smoothing```: Proporsi variansi terbesar dari seluruh variabel yang digunakan untuk modeling 

Keterangan lebih lanjut: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html

### Penerapan Model Gaussian Naive Bayes

In [ ]:
# load modul GaussianNB
from sklearn.naive_bayes import GaussianNB

# inisiasi model
model_nb_gaussian = GaussianNB()

# fitting model
model_nb_gaussian.fit(X_train, y_train)

# buat hasil prediksi
hasil_prediksi = model_nb_gaussian.predict(X_test)

# lihat hasil prediksi
hasil_prediksi

**Model Evaluation**

**Cek Akurasi**

In [ ]:
accuracy_score(y_test, hasil_prediksi)

**Catatan** 

*Accuracy (Akurasi)*

Akurasi adalah salah satu metrik yang biasa digunakan dalam menghitung seberapa baik model yang dibuat dalam memprediksi.

$$Accuracy = \frac{True Positive + True Negative}{True Positive + False Positive + True Negative + False Negative}$$

**Confusion Matrix**

In [ ]:
confusion_matrix(y_test, hasil_prediksi)

**Catatan**

*Confusion Matrix*

Confusion Matrix digunakan untuk melakukan tabulasi silang terhadap hasil antara nilai sebenarnya dengan nilai prediksi.


```python
array([[66,  5],
       [ 6, 36]])
```

Maka:

- True Negative: 66
- False Positive: 5
- False Negative: 6
- True Positive: 36

**Classification Report**

In [ ]:
pd.DataFrame(classification_report(y_test, hasil_prediksi, target_names=['0','1'], output_dict=True))

**Catatan**

![Precision Recall](./images/meme-1.jpg)

*precision*

precision adalah

$$\frac{True Positive}{True Positive + False Positive}$$

*recall* (atau Sensitivity)

recall adalah

$$\frac{True Positive}{True Positive + False Negative}$$

*f1-score*

f1-score adalah 

$$2\times(\frac{precision \times recall}{precision+recall})$$

Makin besar nilainya makin baik.





In [ ]:
y_true = [0, 0, 0, 1, 1, 1, 1, 1]
y_pred = [0, 1, 0, 1, 0, 1, 0, 1]
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
tn, fp, fn, tp

In [ ]:
confusion_matrix(y_true, y_pred)

**Model Inspection**

In [ ]:
pd.DataFrame({
    "Kelas": [0,1],
    "Probabilitas": model_nb_gaussian.class_prior_
}).plot.bar('Kelas','Probabilitas');

In [ ]:
pd.DataFrame({
    "Fitur": training_data.iloc[:,1:].columns,
    "Rerata fitur terhadap kelas Malignant": model_nb_gaussian.theta_[1]
}).sort_values(by='Rerata fitur terhadap kelas Malignant', ascending=False)

**Receiver Operating Characteristic (ROC) & Area Under Curve (AUC)**

In [ ]:
visualizer = ROCAUC(model_nb_gaussian, classes=['Not Malignant', 'Malignant'])
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.finalize()

### Catatan Terkait ROC-AUC

ROC adalah suatu plot yang terdiri dari nilai True Positive Rate (Sensitivity) dan False Positive Rate, dimana

$$False Positive Rate = \frac{False Positive}{False Positive + False Negative}$$

Sedangkan AUC adalah luas area dibawah kurva ROC. AUC dapat dipakai untuk mengukur seberapa baik performa dari model. Semakin mendekati 1 semakin baik.


**Precision-Recall Curve**

In [ ]:
viz = PrecisionRecallCurve(model_nb_gaussian)
viz.fit(X_train, y_train)
viz.score(X_test, y_test)
viz.finalize()

**Catatan**

*Precision-Recall Curve*

*Precision-Recall Curve* adalah kurva yang dibentuk dari nilai precision dan recall. Biasanya dipakai untuk menentukan seberapa baik performa dari model terutama jika terjadinya imbalance class.

*Average Precision* adalah salah satu cara untuk untuk menghitung seberapa baik performa dari model jika kita menggunakan Precision-Recall Curve. Average Precision dirumuskan sebagai berikut:

$$Average Precision = \Sigma_{n}(Recall_{n} - Recall_{n-1}) \times Precision_{n}$$


### Complement Naive Bayes

Complement Naive Bayes mirip dengan multinomial naive Bayes, hanya saja digunakan untuk kasus dimana data memiliki kelas target yang tidak imbang (*imbalance*).

**Kapan Menggunakan Complement Naive Bayes**

- Ketika kelas target tidak seimbang

**Syntax ComplementNB**

```python
ComplementNB(alpha=1.0, fit_prior=True, class_prior=None, norm=False)
```

Keterangan: 

- ```alpha```: Penggunaan smoothing parameter. Berfungsi untuk mencegah nilai probabilitas bernilai 0 karena variabel yang dipakai model berbeda dengan data aslinya.
- ```fit_prior```: Hanya digunakan jika terdapat 1 kelas saja pada kelas target.
- ```class_prior```: Probabilitas prior untuk tiap kelas target.

Keterangan lebih lanjut: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.ComplementNB.html

### Penerapan Model Naive Bayes

In [ ]:
# load modul GaussianNB
from sklearn.naive_bayes import ComplementNB

# inisiasi model
model_nb_complement = ComplementNB(fit_prior=False)

# fitting model
model_nb_complement.fit(X_train, y_train)

# buat hasil prediksi
hasil_prediksi = model_nb_complement.predict(X_test)

# lihat hasil prediksi
hasil_prediksi

**Evaluasi Hasil Prediksi**

**Cek Akurasi**

In [ ]:
accuracy_score(y_test, hasil_prediksi)

**Confusion Matrix**

In [ ]:
confusion_matrix(y_test, hasil_prediksi)

**Classification Report**

In [ ]:
pd.DataFrame(classification_report(y_test, hasil_prediksi, target_names=['0','1'], output_dict=True))

**Model Inspection**

In [ ]:
pd.DataFrame({
    "Variabel": training_data.iloc[:,1:].columns,
    "Weight": model_nb_complement.coef_[0]
}).sort_values(by='Weight', ascending=False)

In [ ]:
pd.DataFrame({
    "Variabel":training_data.iloc[:,1:].columns,
    "Empirical Weight": model_nb_complement.feature_log_prob_[1]
}).sort_values(by='Empirical Weight', ascending=False)

**Receiver Operating Characteristic (ROC) & Area Under Curve (AUC)**

In [ ]:
visualizer = ROCAUC(model_nb_complement, classes=['Not Malignant', 'Malignant'])
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.finalize()

**Precision-Recall Curve**

In [ ]:
viz = PrecisionRecallCurve(model_nb_complement)
viz.fit(X_train, y_train)
viz.score(X_test, y_test)
viz.finalize()

## Logistic Regression



![Logistic Regression](./images/logistic_regression.png)

Sumber Gambar: https://medium.com/@ODSC/logistic-regression-with-python-ede39f8573c7

### Dasar Teori

Logistic Regression adalah salah satu model parametrik yang menggunakan fungsi logistik untuk melakukan fitting pada data:

$$f(\bar{x},\bar{w}) = \frac{1}{1+e^{-\bar{x}\bar{w}}}$$


**Kapan Menggunakan Logistic Regression**

- Jika data yang dipakai untuk pemodelan tidak terlalu besar
- Sebagai base model

**Syntax LogisticRegression**

```python
LogisticRegression(penalty=’l2’, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver=’warn’, max_iter=100, multi_class=’warn’, verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
```

Keterangan:

- ```penalty```: Pemilihan fungsi norm yang digunakan untuk *penalization*. Pilihannya adalah ```l1```, ```l2```, ```elasticnet``` dan ```none```
- ```dual```: Formulasi "dual" atau "primal".
- ```tol```: Nilai toleransi error
- ```C```: Berfungsi untuk *regularization* untuk menghindari kasus *overfitting*
- ```fit_intercept```: Apakah menggunakan *intercept* atau tidak.
- ```class_weight```: Memberikan pembobotan pada kelas target.
- ```random_state```: Berfungsi untuk melakukan seeding pada RNG.
- ```solver```: Algoritma yang digunakan untuk fitting model.
    - Gunakan ‘liblinear’ untuk dataset kecil, gunakan ‘sag’ dan ‘saga’ untuk data ukuran besar.
    - Untuk kasus multiclass, gunakan ‘newton-cg’, ‘sag’, ‘saga’ dan ‘lbfgs’ untuk multinomial.
    - ‘newton-cg’, ‘lbfgs’, ‘sag’ dan ‘saga’ dapat melakukan regularisasi L2 atau tanpa penalti
    - ‘liblinear’ dan ‘saga’ dapat melakukan regularisasi menggunakan L1
    - ‘saga’ dapat menggunakan regularisasi menggunakan ‘elasticnet’
    - ‘liblinear’ harus menggunakan penalti
- ```max_iter```: Iterasi maksimal
- ```multi_class```: Apakah kasusnya binary atau multiclass.
    - Gunakan ‘ovr’ untuk kasus binary
    - Gunakan ‘multinomial’ untuk kasus multiclass
    - Gunakan 'auto' untuk memilih secara otomatis tergantung dari kelas targetnya
- ```n_jobs```: Jumlah core CPU yang digunakan untuk pemodelan

Sisanya dapat dibaca di: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

### Penerapan Model Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

model_logistic_reg = LogisticRegression(random_state=1000, solver='liblinear', multi_class='ovr')

In [ ]:
# fitting model
model_logistic_reg.fit(X_train, y_train)

# buat hasil prediksi
hasil_prediksi = model_logistic_reg.predict(X_test)

# lihat hasil prediksi
hasil_prediksi

**Evaluasi Model**

**Akurasi**

In [ ]:
accuracy_score(y_test, hasil_prediksi)

**Confusion Matrix**

In [ ]:
confusion_matrix(y_test, hasil_prediksi)

**Classification Report**

In [ ]:
pd.DataFrame(classification_report(y_test, hasil_prediksi, output_dict=True))

**Model Inspection**

**Coefficients Weight**

In [ ]:
nama_feature = training_data.iloc[:,1:].columns.tolist()

In [ ]:
eli5.explain_weights(model_logistic_reg, feature_names=nama_feature)

**Receiver Operating Characteristic (ROC) & Area Under Curve (AUC)**

In [ ]:
visualizer = ROCAUC(model_logistic_reg, classes=['Not Malignant', 'Malignant'])
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.finalize()

**Precision-Recall Curves**

In [ ]:
viz = PrecisionRecallCurve(model_logistic_reg)
viz.fit(X_train, y_train)
viz.score(X_test, y_test)
viz.finalize()

## K-Nearest Neighbors

![KNN](./images/knn_.png)

Sumber Gambar: https://www.kdnuggets.com/2019/07/classifying-heart-disease-using-k-nearest-neighbors.html

### Dasar Teori

K-Nearest Neighbors (KNN) adalah salah satu model machine learning non-parametrik dimana model menggunakan posisi dari tiap titik data pada saat training untuk mengklasifikasikan data yang baru.

Karena posisi titik sangat diperhitungkan, maka KNN bergantung kepada fungsi jarak yang digunakan dalam pemodelan, diantaranya:

**Jarak Manhatan (*Manhatan Distance*)**

$$d(x,y) = \Sigma_{i=1}^{k} |x_{i} - y_{i}|$$

**Jarak Euclidean (*Euclidean Distance*)**

$$d(x,y) = \sqrt{\Sigma_{i=1}^{k} (x_{i} - y_{i})^{2}}$$

**Jarak Minkowski (*Minkowski Distance*)**

$$d(x,y) = (\Sigma_{i=1}^{k} (x_{i} - y_{i})^{p})^{p}$$

**Kapan Menggunakan K-Nearest Neighbors**

- Base model

**Syntax KNeighborsClassifier**

```python
KNeighborsClassifier(n_neighbors=5, weights=’uniform’, algorithm=’auto’, p=2, metric=’minkowski’, n_jobs=None)
```

Keterangan: 

- ```n_neigbors```: Jumlah titik yang digunakan untuk menentukan kelas dari data yang baru.
- ```weights```: Pembobotan yang dipakai untuk melakukan prediksi. Terdapat 2 pilihan, yaitu:
    - ```uniform```: Setiap titik diperlakukan sama
    - ```distance```: Setiap titik diberi bobot berdasarkan inverse jarak.
- ```algorithm```: Algoritma yang dipakai untuk menghitung nilai ketetanggaan antar titik. Terdapat 4 pilihan:
    - ```balll_tree```
    - ```kd_tree```
    - ```brute```
    - ```auto```
- ```p```: Derajat atau pangkat yang digunakan untuk menghitung jarak antar titik berdasarkan rumus jarak Minkowski. 
    - Jika ```p=1```, maka jarak yang digunakan jarak Manhatan
    - Jika ```p=2```, maka jarak yang digunakan jarak Euclidean
- ```metric```: Fungsi jarak yang dipakai untuk menghitung jarak antar titik
- ```n_jobs```: Berapa banyak CPU yang digunakan untuk membuat model.

**Penerapan Model K-Nearest Neighbors**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier(n_neighbors=3, algorithm='auto')

model_knn.fit(X_train, y_train)

hasil_prediksi = model_knn.predict(X_test)

hasil_prediksi

**Evaluasi**

In [ ]:
accuracy_score(y_test, hasil_prediksi)

In [ ]:
confusion_matrix(y_test, hasil_prediksi)

In [ ]:
pd.DataFrame(
classification_report(y_test, hasil_prediksi, output_dict=True)
)

**Model Inspection**

**Receiver Operating Characteristic (ROC) & Area Under Curve (AUC)**

In [ ]:
visualizer = ROCAUC(model_knn, classes=['Not Malignant', 'Malignant'])
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.finalize()

**Precission-Recall Curve**

In [ ]:
viz = PrecisionRecallCurve(model_knn)
viz.fit(X_train, y_train)
viz.score(X_test, y_test)
viz.finalize()

## Decision Tree

![Decision Tree](./images/decision_tree.png)

Sumber Gambar: https://www.displayr.com/what-is-a-decision-tree/

### Dasar Teori

Decision Tree adalah salah satu model machine learning non parametrik yang didasarkan pada pembentukan pohon keputusan. Model ini dibangun dengan cara menentukan root node, lalu membentuk cabang-cabang keputusan yang dibuat berdasarkan ukuran tertentu yang disebut sebagai *split criteria* atau *impurity measures*. Hasil yang diperoleh dari decision tree bisa berupa aturan-aturan yang dapat diinterpretasikan ke dalam bahasa manusia sehingga model decision tree lebih mudah dipahami oleh orang awam sekalipun.

**Beberapa *Split Criteria/Impurity Measures***

![Impurity Measures](./images/impurity.png)


**Kapan Menggunakan Decision Tree**

- Ketika membutuhkan model yang memiliki performa lebih baik dibanding base model dan mudah diinterpretasikan
- Ketika masalah yang ingin diselesaikan adalah masalah bisnis dimana membutuhkan panduan dalam membuat keputusan

**Syntax DecisionTreeClassifier**

```python
DecisionTreeClassifier(criterion=’gini’, splitter=’best’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)
```

Keterangan:

- ```criterion```: ukuran yang dipakai untuk melakukan split pada node, terdapat dua pilihan yaitu 'gini' untuk penggunaan gini index dan 'entropy' untuk penggunaan information gain.
- ```splitter```: strategi yang digunakan untuk split pada tiap node, terdapat dua pilihan yaitu 'best' dan 'random'.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model_decision_tree = DecisionTreeClassifier(random_state=1000)

model_decision_tree.fit(X_train, y_train)

hasil_prediksi = model_decision_tree.predict(X_test)

hasil_prediksi

**Model Evaluation**

In [ ]:
accuracy_score(y_test, hasil_prediksi)

In [ ]:
confusion_matrix(y_test, hasil_prediksi)

In [ ]:
pd.DataFrame(
classification_report(y_test, hasil_prediksi, output_dict=True)
)

**Model Inspection**

**Feature Importance & Rules**

In [ ]:
eli5.sklearn.explain_decision_tree(model_decision_tree, feature_names=training_data.iloc[:,1:].columns.tolist())

**Tree Visualization**

In [ ]:
import graphviz
dot_data = tree.export_graphviz(model_decision_tree, out_file=None, filled=True, rounded=True, special_characters=True)
graph = graphviz.Source(dot_data)
graph

**Receiver Operating Characteristic (ROC) & Area Under Curve (AUC)**

In [ ]:
visualizer = ROCAUC(model_decision_tree, classes=['Not Malignant', 'Malignant'])
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.finalize()

**Precission-Recall Curve**

In [ ]:
viz = PrecisionRecallCurve(model_decision_tree)
viz.fit(X_train, y_train)
viz.score(X_test, y_test)
viz.finalize()

## AdaBoost

![AdaBoost](./images/adaboost.png)

Sumber Gambar: https://medium.com/swlh/boosting-and-bagging-explained-with-examples-5353a36eb78d


**Kapan Menggunakan AdaBoost**

- Ketika akurasi lebih diutamakan dibanding interpretabilitas.
- Mencari tahu mana variabel yang lebih penting dibanding variabel lainnya menggunakan feature importance.

**Syntax AdaBoostClassifier**

```python
AdaBoostClassifier(base_estimator=None, n_estimators=50, learning_rate=1.0, algorithm=’SAMME.R’, random_state=None)
```

Keterangan:

- ```base_estimator```: Estimator yang digunakan untuk membuat model.
- ```n_estimators```: Banyak estimator yang dibuat.
- ```learning_rate```: Berfungsi untuk memperkecil kontribusi tiap estimator yang dibuat.
- ```algorithm```: Algoritma yang digunakan untuk. Terdapat dua pilihan, yaitu 'SAMME' dan 'SAMME.R'
- ```random_state```: Berfungsi untuk seeding random number generator

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

model_adaboost = AdaBoostClassifier(random_state=1000)

model_adaboost.fit(X_train, y_train)

hasil_prediksi = model_adaboost.predict(X_test)

hasil_prediksi

**Model Evaluation**

In [ ]:
accuracy_score(y_test, hasil_prediksi)

In [ ]:
confusion_matrix(y_test, hasil_prediksi)

In [ ]:
pd.DataFrame(
classification_report(y_test, hasil_prediksi, output_dict=True)
)

**Model Inspection**

**Feature Importance**

In [ ]:
eli5.explain_weights(model_adaboost, feature_names=training_data.iloc[:,1:].columns.tolist())

**Receiver Operating Characteristic (ROC) & Area Under Curve (AUC)**

In [ ]:
visualizer = ROCAUC(model_adaboost, classes=['Not Malignant', 'Malignant'])
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.finalize()

**Precision-Recall Curve**

In [ ]:
viz = PrecisionRecallCurve(model_adaboost)
viz.fit(X_train, y_train)
viz.score(X_test, y_test)
viz.finalize()

## Random Forest

![Random Forest](./images/random_forest.png)

Sumber gambar: https://www.machinehack.com/course/machinehack-practise-5-random-forest-regression/

**Kapan Menggunakan Random Forest**

**Syntax RandomForest**

```python
RandomForestClassifier(n_estimators=’warn’, criterion=’gini’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None)
```

Keterangan:

- ```n_estimators```: Jumlah pohon yang dibuat.
- ```criterion```: Kriteria yang digunakan untuk melakukan split pada node. Terdapat dua pilihan, diantaranya 'gini' untuk penggunaan gini index dan 'entropy' untuk penggunaan information gain.
- ```max_depth```: Seberapa banyak node yang akan dibentuk. Semakin besar maka semakin dalam pohon yang dibuat.
- ```min_sample_split```: Jumlah sampel minimum yang digunakan node untuk split.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_randomforest = RandomForestClassifier(n_estimators=100, random_state=1000, max_depth=4)

model_randomforest.fit(X_train, y_train)

hasil_prediksi = model_randomforest.predict(X_test)

hasil_prediksi

**Model Evaluation**

In [ ]:
accuracy_score(y_test, hasil_prediksi)

In [ ]:
confusion_matrix(y_test, hasil_prediksi)

In [ ]:
pd.DataFrame(
classification_report(y_test, hasil_prediksi, output_dict=True)
)

**Model Inspection**

**Feature Importance**

In [ ]:
eli5.sklearn.explain_rf_feature_importance(model_randomforest, feature_names=training_data.iloc[:,1:].columns.tolist())

**Receiver Operating Characteristic (ROC) & Area Under Curve (AUC)**

In [ ]:
visualizer = ROCAUC(model_randomforest, classes=['Not Malignant', 'Malignant'])
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.finalize()

**Precision-Recall Curve**

In [ ]:
viz = PrecisionRecallCurve(model_randomforest)
viz.fit(X_train, y_train)
viz.score(X_test, y_test)
viz.finalize()

## Gradient Boosting Tree

![Gradient Boosting Tree](./images/gradient_boosting_tree.png)

Sumber gambar: https://bradleyboehmke.github.io/HOML/gbm.html

**Kapan Menggunakan Gradient Boosting Tree**

**Syntax GradientBoostingClassifier**

```python
GradientBoostingClassifier(loss=’deviance’, learning_rate=0.1, n_estimators=100, subsample=1.0, criterion=’friedman_mse’, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, presort=’auto’, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001)
```

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model_gradient_boosting = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=1000)

model_gradient_boosting.fit(X_train, y_train)

hasil_prediksi = model_gradient_boosting.predict(X_test)

hasil_prediksi

**Model Evaluation**

In [ ]:
accuracy_score(y_test, hasil_prediksi)

In [ ]:
confusion_matrix(y_test, hasil_prediksi)

In [ ]:
pd.DataFrame(
classification_report(y_test, hasil_prediksi, output_dict=True)
)

**Model Inspection**

In [ ]:
eli5.explain_weights(model_gradient_boosting, feature_names=training_data.iloc[:,1:].columns.tolist())

Receiver Operating Characteristic (ROC) & Area Under Curve (AUC)

In [ ]:
visualizer = ROCAUC(model_gradient_boosting, classes=['Not Malignant', 'Malignant'])
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.finalize()

**Precision-Recall Curve**

In [ ]:
viz = PrecisionRecallCurve(model_randomforest)
viz.fit(X_train, y_train)
viz.score(X_test, y_test)
viz.finalize()

## Support Vector Machine

![SVM](./images/svm.png)

Sumber gambar: https://www.datacamp.com/community/tutorials/svm-classification-scikit-learn-python

**Kapan Menggunakan SVM**

**Syntax SVC**

```python
SVC(C=1.0, kernel=’rbf’, degree=3, gamma=’auto_deprecated’, coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape=’ovr’, random_state=None)
```



In [ ]:
from sklearn.svm import SVC

model_svm = SVC(kernel='linear', gamma = 'scale', random_state=1000, probability=True)

model_svm.fit(X_train, y_train)

hasil_prediksi = model_svm.predict(X_test)

hasil_prediksi

**Model Evaluation**

In [ ]:
accuracy_score(y_test, hasil_prediksi)

In [ ]:
confusion_matrix(y_test, hasil_prediksi)

In [ ]:
pd.DataFrame(
classification_report(y_test, hasil_prediksi, output_dict=True)
)

**Model Inspection**

In [ ]:
eli5.explain_weights(model_svm, feature_names=nama_feature)

In [ ]:
visualizer = ROCAUC(model_svm, classes=['Not Malignant', 'Malignant'])
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.finalize()

In [ ]:
viz = PrecisionRecallCurve(model_randomforest)
viz.fit(X_train, y_train)
viz.score(X_test, y_test)
viz.finalize()